In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0,"..")

In [11]:
from utils.construct_model import LoadSupervisedModel

In [12]:
path_to_pretrained_weights = "/project/dane2/wficai/BenchMD/models/pretrained/supervised/radimagenet/checkpoint-159.pth.tar"
model = LoadSupervisedModel(arch="resnet50")
model.load_pretrained_weights("RadImageNet", path_to_pretrained_weights)
model.modify_head(num_classes=10)
model.freeze_backbone()
model.produce_model()


Initializing ResNet50 Model with RadImageNet Weights


/home/jplineb/VICReg/vicreg_mod/notebooks/../utils/construct_model.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state_dict = torch.load(pretrained_path, 

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

# Use model comparison code

In [4]:
from model_layer_comparison import load_model


/home/jplineb/.conda/envs/disres/lib/python3.11/site-packages/torchxrayvision/utils.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
# for (name1, param1) in model.named_parameters():
#     if 'bn' in name1.lower():
#         print(f"Parameter: {name1}: ")
        

In [5]:

path_to_pretrained_weights_1 = "/home/jplineb/VICReg/vicreg_mod/checkpoint/bcn_20000_VICReg_ImageNet_20250410_173512/checkpoint_20250410_epoch15.pth"
path_to_pretrained_weights_2 = "/home/jplineb/VICReg/vicreg_mod/checkpoint/chexpert_Supervised_ImageNet_20250411_114359/checkpoint_20250411_epoch4.pth"
model_1 = load_model(path_to_pretrained_weights_1)
model_2 = load_model(path_to_pretrained_weights_2)
print(model_1)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
import torch

def compare_bn_layers(model1, model2):
    """
    Compare BatchNorm layer weights between two models with more detailed information.
    """
    bn_diffs = []
    
    # Get all named parameters from both models
    params1 = dict(model1.named_parameters())
    params2 = dict(model2.named_parameters())
    
    # Find all BatchNorm parameters
    for name, param1 in params1.items():
        if 'bn' in name.lower() and name in params2:
            param2 = params2[name]
            
            # Calculate statistics for both parameters
            stats1 = {
                'mean': param1.mean().item(),
                'std': param1.std().item(),
                'min': param1.min().item(),
                'max': param1.max().item(),
                'dtype': param1.dtype,
                'shape': param1.shape
            }
            
            stats2 = {
                'mean': param2.mean().item(),
                'std': param2.std().item(),
                'min': param2.min().item(),
                'max': param2.max().item(),
                'dtype': param2.dtype,
                'shape': param2.shape
            }
            
            # Calculate absolute difference between parameters
            diff = torch.abs(param1 - param2).mean().item()
            
            print(f"\nLayer: {name}")
            print(f"Model 1 stats: {stats1}")
            print(f"Model 2 stats: {stats2}")
            print(f"Absolute difference: {diff}")
            
            bn_diffs.append((name, diff, stats1, stats2))
            
    return bn_diffs

def check_model_loading(model, checkpoint_path):
    print(f"\n=== Checking model loaded from {checkpoint_path} ===\n")
    
    # 1. Load the checkpoint directly
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    print("1. Checkpoint Structure:")
    if isinstance(checkpoint, dict):
        print("Checkpoint keys:", checkpoint.keys())
        # If there's a 'model' or 'state_dict' key, show its structure
        if 'model' in checkpoint:
            print("\n'model' key structure:")
            for key in checkpoint['model'].keys():
                if 'bn' in key.lower():
                    print(f"  {key}: shape={checkpoint['model'][key].shape}, dtype={checkpoint['model'][key].dtype}")
        elif 'state_dict' in checkpoint:
            print("\n'state_dict' key structure:")
            for key in checkpoint['state_dict'].keys():
                if 'bn' in key.lower():
                    print(f"  {key}: shape={checkpoint['state_dict'][key].shape}, dtype={checkpoint['state_dict'][key].dtype}")
    else:
        print("Checkpoint is not a dictionary")
    
    # 2. Check the model's BatchNorm layers
    print("\n2. Model BatchNorm layers after loading:")
    model_bn_stats = {}
    for name, param in model.named_parameters():
        if 'bn' in name.lower():
            model_bn_stats[name] = {
                'dtype': param.dtype,
                'shape': param.shape,
                'mean': param.data.mean().item(),
                'std': param.data.std().item(),
                'min': param.data.min().item(),
                'max': param.data.max().item()
            }
            print(f"\n{name}:")
            for stat_name, stat_value in model_bn_stats[name].items():
                print(f"  {stat_name}: {stat_value}")
    
    # 3. Compare checkpoint and model values
    print("\n3. Comparing checkpoint and model values:")
    if isinstance(checkpoint, dict):
        # Get the actual state dict from the checkpoint
        if 'model' in checkpoint:
            ckpt_state_dict = checkpoint['model']
        elif 'state_dict' in checkpoint:
            ckpt_state_dict = checkpoint['state_dict']
        else:
            ckpt_state_dict = checkpoint
            
        for name, param in model.named_parameters():
            if 'bn' in name.lower():
                if name in ckpt_state_dict:
                    ckpt_param = ckpt_state_dict[name]
                    print(f"\n{name}:")
                    print(f"  Checkpoint - mean: {ckpt_param.mean().item():.6f}, std: {ckpt_param.std().item():.6f}")
                    print(f"  Model     - mean: {param.data.mean().item():.6f}, std: {param.data.std().item():.6f}")
                    print(f"  Difference - mean: {abs(ckpt_param.mean() - param.data.mean()).item():.6f}")
                else:
                    print(f"\n{name}: Not found in checkpoint!")
    
    # 4. Check if model is in eval mode
    print("\n4. Model state:")
    print(f"  Training mode: {model.training}")
    print(f"  Device: {next(model.parameters()).device}")
    
    return model_bn_stats
            

In [7]:
check_model_loading(model_1, path_to_pretrained_weights_1)


=== Checking model loaded from /home/jplineb/VICReg/vicreg_mod/checkpoint/bcn_20000_VICReg_ImageNet_20250410_173512/checkpoint_20250410_epoch15.pth ===

1. Checkpoint Structure:
Checkpoint keys: dict_keys(['epoch', 'all_auc', 'best_auc', 'model', 'optimizer', 'scheduler'])

'model' key structure:
  0.bn1.weight: shape=torch.Size([64]), dtype=torch.float32
  0.bn1.bias: shape=torch.Size([64]), dtype=torch.float32
  0.bn1.running_mean: shape=torch.Size([64]), dtype=torch.float32
  0.bn1.running_var: shape=torch.Size([64]), dtype=torch.float32
  0.bn1.num_batches_tracked: shape=torch.Size([]), dtype=torch.int64
  0.layer1.0.bn1.weight: shape=torch.Size([64]), dtype=torch.float32
  0.layer1.0.bn1.bias: shape=torch.Size([64]), dtype=torch.float32
  0.layer1.0.bn1.running_mean: shape=torch.Size([64]), dtype=torch.float32
  0.layer1.0.bn1.running_var: shape=torch.Size([64]), dtype=torch.float32
  0.layer1.0.bn1.num_batches_tracked: shape=torch.Size([]), dtype=torch.int64
  0.layer1.0.bn2.wei

{'bn1.weight': {'dtype': torch.float32,
  'shape': torch.Size([64]),
  'mean': 6.40048885345459,
  'std': 3.816087007522583,
  'min': 0.04330281540751457,
  'max': 22.6646785736084},
 'bn1.bias': {'dtype': torch.float32,
  'shape': torch.Size([64]),
  'mean': 5.257490158081055,
  'std': 9.832067489624023,
  'min': -33.26302719116211,
  'max': 38.07280731201172},
 'layer1.0.bn1.weight': {'dtype': torch.float32,
  'shape': torch.Size([64]),
  'mean': 5.749851703643799,
  'std': 2.5768628120422363,
  'min': 2.377289056777954,
  'max': 11.615592956542969},
 'layer1.0.bn1.bias': {'dtype': torch.float32,
  'shape': torch.Size([64]),
  'mean': 2.2531023025512695,
  'std': 4.957939147949219,
  'min': -5.798407554626465,
  'max': 23.544918060302734},
 'layer1.0.bn2.weight': {'dtype': torch.float32,
  'shape': torch.Size([64]),
  'mean': 4.5329108238220215,
  'std': 2.4394595623016357,
  'min': 2.299187660217285,
  'max': 13.980375289916992},
 'layer1.0.bn2.bias': {'dtype': torch.float32,
  'sha